In [1]:
import glob

import pandas as pd

# importing user-defined functions from udf_eda.py
from src import udf_eda as udf

pd.set_option("display.float_format", lambda x: f"{x:5f}")


In [2]:
file_names = [x.replace("\\", "/") for x in glob.glob("./data/datasets/**/ODP *.xlsx")]
df_BHV_CFF_orig, df_NF_orig, df_EXT_orig, df_batches_orig = udf.read_bind(file_names)
_, _, _, df_features_orig = udf.read_bind(
    [x.replace("\\", "/") for x in glob.glob("./data/ODP *.xlsx")]
)
print(f"#Available Batches: {len(file_names)}")


100%|██████████| 109/109 [03:25<00:00,  1.89s/it]


The following batches have incompatible data:  ['100001576', '100001778']
# of batches read:  101
Missing batches, if any: {'', 'FF_N'}
How many NaN values exist in the merged data:  8936962
Shape of the merged data:  (87436, 364)


100%|██████████| 1/1 [00:01<00:00,  1.70s/it]

The following batches have incompatible data:  []
# of batches read:  1
Missing batches, if any: {''}
How many NaN values exist in the merged data:  1187
Shape of the merged data:  (1190, 169)
#Available Batches: 109


In [3]:
df_BHV_CFF = df_BHV_CFF_orig.copy()
df_NF = df_NF_orig.copy()
df_EXT = df_EXT_orig.copy()
df_batches = df_batches_orig.copy()
df_features = df_features_orig.copy()


In [4]:
df_features.drop("TAG", axis=1, inplace=True)
# remove blank columns that have been named "Unnamed*" by pandas when reading the files
df_batches.drop(
    [x for x in df_batches.columns if x.startswith("Unnamed")], inplace=True
)
print(f"Amount of NaN in the data: {df_batches.isna().sum().sum()}")
print(f"Dataframe shape: {df_batches.shape}")


Amount of NaN in the data: 8936962
Dataframe shape: (87436, 364)


In [5]:
# restrict columns to the ones selected by the company
df_batches = df_batches[df_features.columns.tolist()]
# remove specific batch
df_batches = df_batches[df_batches["id"] != "100001510"]
print(f"Amount of NaN in the data: {df_batches.isna().sum().sum()}")
print(f"Dataframe shape: {df_batches.shape}")
print(f"Columns with NA values: {df_batches.columns[df_batches.isna().any()].tolist()}")


Amount of NaN in the data: 102460
Dataframe shape: (86278, 168)
Columns with NA values: ['118FIC606', '118ZLH303', '118LS960', '118LS690', '118ZLL417', '118ZLL427', '118ZLL437', '118ZLL447', '118FI912', '118FI913', '118PI629', '118PI628', '118PI924', '118PI964', '118PI925', '118PI639', '118PI638', '118PI934', '118PI952', '118PI935', '118PI649', '118PI648', '118PI944', '118PI953', '118PI945']


In [6]:
# drop columns containing NA values
df_batches.drop(
    df_batches.columns[df_batches.isna().any()].tolist(), axis=1, inplace=True
)
print(f"Dataframe shape: {df_batches.shape}")


Dataframe shape: (86278, 143)


In [7]:
# drop "object" (str) columns
df_batches.drop(
    df_batches.select_dtypes(include=["O"]).columns.tolist(), axis=1, inplace=True
)
# drop columns starting with 164
df_batches.drop(df_batches.filter(regex="^164").columns.tolist(), axis=1, inplace=True)
print(f"Dataframe shape: {df_batches.shape}")


Dataframe shape: (86278, 123)


In [8]:
# check which sensors are available for units from 158 to 165
units_158_to_165_sensors = set()
for unit in range(158, 166):
    for x in df_batches.filter(regex=f"^{unit}").columns.tolist():
        units_158_to_165_sensors.add(x.replace(f"{unit}", ""))
print(units_158_to_165_sensors)

# check which units are available for each sensor
units_per_sensor = dict()
for sensor in units_158_to_165_sensors:
    units_per_sensor[sensor] = [
        x.replace(sensor, "") for x in df_batches.filter(regex=sensor).columns
    ]
print(units_per_sensor)


# keep only valid sensors according to TI
def filtering(row, sensor, units):
    i = 0
    row_sum = 0
    for unit in units:
        if row[f"{unit}TI650"] < 30:
            i += 1
            row_sum += row[f"{unit}{sensor}"]
    if i > 0:
        return row_sum / i
    else:
        return 0


for sensor, units in units_per_sensor.items():
    df_batches[sensor] = df_batches.agg(filtering, axis=1, sensor=sensor, units=units)
print(f"Dataframe shape pre-drop: {df_batches.shape}")

# drop units measurements as we have the mean ones
for unit in range(158, 166):
    df_batches.drop(
        df_batches.filter(regex=f"^{unit}").columns.tolist(), axis=1, inplace=True
    )
print(f"Dataframe shape post-drop: {df_batches.shape}")


{'FI679P', 'PI678', 'TI670', 'FI667', 'FI654', 'TI684', 'TI650', 'FI652', 'CAL4552', 'CAL4551', 'FI656', 'FI666'}
{'FI679P': ['158'], 'PI678': ['158'], 'TI670': ['158'], 'FI667': ['158', '159', '160', '161', '162', '163'], 'FI654': ['158', '159', '160', '161', '162', '163'], 'TI684': ['158'], 'TI650': ['158', '159', '160', '161', '162', '163', '165'], 'FI652': ['158', '159', '160', '161', '162', '163'], 'CAL4552': ['158', '159', '160', '161', '162', '163', '165'], 'CAL4551': ['158', '159', '160', '161', '162', '163', '165'], 'FI656': ['158', '159', '161', '162', '163'], 'FI666': ['158', '159', '161', '162', '163']}
Dataframe shape pre-drop: (86278, 135)
Dataframe shape post-drop: (86278, 82)


In [9]:
# check which sensors are available for units 107 and 108
units_107_108_sensors = set()
for unit in range(107, 109):
    for x in df_batches.filter(regex=f"^{unit}").columns.tolist():
        units_107_108_sensors.add(x.replace(f"{unit}", ""))
print(units_107_108_sensors)


# keep only valid sensors according to PI
def filtering(row, sensor, units):
    i = 0
    row_sum = 0
    for unit in units:
        if f"{unit}{sensor}" in row and row[f"{unit}PI650"] > 10:
            i += 1
            row_sum += row[f"{unit}{sensor}"]
    if i > 0:
        return row_sum / i
    else:
        return 0


for sensor in units_107_108_sensors:
    df_batches[sensor] = df_batches.agg(
        filtering, axis=1, sensor=sensor, units=[107, 108]
    )
print(f"Dataframe shape pre-drop: {df_batches.shape}")

# drop units measurements as we have the mean ones
for unit in range(107, 109):
    df_batches.drop(
        df_batches.filter(regex=f"^{unit}").columns.tolist(), axis=1, inplace=True
    )
print(f"Dataframe shapepost-drop: {df_batches.shape}")


{'AI612', 'TI660', 'FI689A', 'TI656', 'AI673A', 'TI607', 'FI657', 'TI664', 'FI677', 'FI681', 'AI610', 'PI650', 'FI693A', 'LI606', 'FI653', 'AI672', 'AI611', 'FI673', 'FI685A', 'FI665', 'TI652', 'AI613', 'AI677A', 'FI669', 'AI674A'}
Dataframe shape pre-drop: (86278, 107)
Dataframe shapepost-drop: (86278, 80)


In [10]:
list_ext = [
    "118PI946",
    "118PI628",
    "118PI649",
    "113LI682.1",
    "118PI952",
    "118PI953",
    "118PI964",
    "118PI925",
    "118PI945",
    "118PI648",
    "118FI912",
    "118PI934",
    "118PI638",
    "118PI935",
    "118PI629",
    "118PI924",
    "118PI639",
    "118FI913",
    "113LI682",
    "118PI944",
    "118TI977",
    "164FI656",
    "164FI666",
    "164FI667",
    "118PI936",
    "118ZLL417",
    "118ZLL427",
    "118AI641",
    "118PI974",
    "118SI643",
    "163TIC650",
    "118SI633",
    "118ZLL437",
    "118PI927",
    "118PI916",
    "118SI613",
    "118LS690",
    "118ZLH303",
    "164PI653",
    "163PI663",
    "118LS960",
    "118PI917",
    "890PI610",
    "118SI623",
    "118PI937",
    "118PI951",
    "118ZLL447",
    "163PI655",
    "118FIC606",
    "118PI947",
    "164PI655",
    "118AI631",
    "118PI954",
    "118PI984",
    "164PI651",
    "118AI621",
    "118PI926",
    "118AI611",
    "164PI663",
    "163TI650",
    "164FI652",
    "164FI654",
]
# drop specific EXT columns
df_EXT.drop(list_ext, axis=1, inplace=True)


In [11]:
# replace invalid EXT data with 0
df_batches.loc[
    df_batches["118CV502"] <= 0,
    df_batches[df_EXT.columns[2:].tolist()].columns.tolist(),
] = 0
df_batches.loc[
    df_batches["118CV501"] <= 0,
    df_batches[df_EXT.columns[2:].tolist()].columns.tolist(),
] = 0


In [12]:
# extract yield from produzione_CStOA_2021_ed12.xlsx
df_yield = pd.read_excel(
    "./data/datasets/produzione_CStOA_2021_ed12.xlsx",
    sheet_name="dati-produzione",
    header=1,
)
df_yield = df_yield[["O.D.P.", "Resa"]]
df_yield.dropna(axis=0, how="any", inplace=True)
df_yield["O.D.P."] = df_yield["O.D.P."].astype(int).astype("string")
df_yield.columns = ["id", "result"]
df_yield.result = round(df_yield.result, 6)


In [13]:
# keep only meaningful part of id
df_batches["id"] = df_batches["id"].str[-4:]
df_yield["id"] = df_yield["id"].str[-4:]


In [15]:
# merge target with data
df_batches = df_yield.merge(df_batches, how="inner")
print(f"Dataframe shape: {df_batches.shape}")


Dataframe shape: (85088, 81)


In [16]:
# add start_date, end_date, processing_time_mins and timestamp_index columns
df_begin_end = (
    df_batches.groupby(["id"])["timeseries"].agg(["min", "max"]).reset_index()
)
df_begin_end.columns = ["id", "start_date", "end_date"]
df_begin_end["processing_time_mins"] = (
    (df_begin_end["end_date"] - df_begin_end["start_date"]) / pd.Timedelta(minutes=5)
) + 1
df_batches = df_begin_end.merge(df_batches, how="right")
df_batches.insert(5, "timestamp_index", df_batches.groupby("id").cumcount())
# adjust DIAFTOTALE values
df_batches.loc[df_batches["DIAFTOTALE"] > 3, "DIAFTOTALE"] = 2.59
df_batches.loc[df_batches["DIAFTOTALE"] < -1, "DIAFTOTALE"] = -0.4
# insert the progress column which measures progress in percentage terms
df_batches.insert(
    6,
    "progress_perc",
    round(
        (df_batches.timestamp_index / df_batches.processing_time_mins) * 100, 0
    ).astype(int),
)


In [17]:
# group every batch into its own progress_perc with mean
df_grouped = (
    df_batches.groupby(
        [
            "id",
            "start_date",
            "end_date",
            "processing_time_mins",
            "result",
            "progress_perc",
        ]
    )[df_batches.columns[8:].tolist()]
    .mean()
    .reset_index()
)

# compute stats using pandas describe
df_stats_describe = df_grouped.groupby(["id"]).describe()
df_stats_describe.columns = [
    "_".join(col).strip() for col in df_stats_describe.columns.values
]
df_stats_describe.reset_index(inplace=True)
df_stats_describe = df_stats_describe[
    df_stats_describe.columns.drop(
        df_stats_describe.filter(regex="_count").columns.tolist()
    )
]


In [18]:
# create df with suspected change points
change_point_vars = df_grouped.columns[5:].insert(0, "id")
df_change_points = df_grouped[
    df_grouped["progress_perc"].isin([0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
]
df_change_points = df_change_points[change_point_vars]
df_melt = pd.melt(df_change_points, id_vars=["id", "progress_perc"])
df_melt["variable"] = (
    df_melt["variable"].astype(str) + "_" + df_melt["progress_perc"].astype(str)
)
df_melt.drop(columns=["progress_perc"], inplace=True)
df_change_points = df_melt.pivot(index="id", columns="variable", values="value")
df_change_points.reset_index(inplace=True)


In [19]:
df_final = df_stats_describe.merge(df_change_points, on="id")
df_final.drop(
    [
        "processing_time_mins_mean",
        "processing_time_mins_std",
        "processing_time_mins_min",
        "processing_time_mins_25%",
        "processing_time_mins_50%",
        "processing_time_mins_75%",
        "processing_time_mins_max",
        "result_mean",
        "result_std",
        "result_min",
        "result_25%",
        "result_50%",
        "result_75%",
        "result_max",
        "progress_perc_mean",
        "progress_perc_std",
        "progress_perc_min",
        "progress_perc_25%",
        "progress_perc_50%",
        "progress_perc_75%",
        "progress_perc_max",
    ],
    axis=1,
    inplace=True,
)
# re-add yield
df_final = df_yield.merge(df_final, how="inner")
df_final.dropna(axis=0, inplace=True)
print(f"Dataframe shape: {df_final.shape}")
print(f"Amount of NaN in the data: {df_final.isna().sum().sum()}")
df_final.to_csv("./data/suanfarma_train_data.csv")


Dataframe shape: (92, 1484)
Amount of NaN in the data: 0
